In [1]:
import json
import logging
import os
import pandas as pd
from tqdm import tqdm
from nltk import ngrams
import multiprocessing as mp
from math import log

In [2]:
#func
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8')
    text = file.read()
    file.close()
    return text


def getUpper(word):
    word = word[0].upper() + word[1:]
    return word


# gender pair list taken from GN-Glove
DEFAULT_MALE_NOUNS = load_doc('male_word_file.txt').split('\n')
DEFAULT_FEMALE_NOUNS = load_doc('female_word_file.txt').split('\n')


def gender_ratios_m_f(data, maleCount, femaleCount):
    scoresP = []
    scoresPC = []
    bias_record = {}
    for words in data:
        if (data[words]['m'] + data[words]['f'] > MinCount):
            if words in occupations:
                score = abs(
                    log((data[words]['m'] + 1) / (data[words]['f'] + 1)))
                scoresP.append(score)

                pm = data[words]['m'] / (maleCount + 0.00001)
                pf = data[words]['f'] / (femaleCount + 0.00001)
                score_c = abs(log((pm + 0.00001) / (pf + 0.00001)))
                scoresPC.append(score_c)

                bias_record[words] = log(
                    (data[words]['m'] + 1) / (data[words]['f'] + 1))
    return scoresP, scoresPC, bias_record


# Taken from jtcrammer repo
def get_cooccurrences(file, data, window):
    with open(file, 'r', encoding='utf-8') as fp:
        sentences = fp.read()
    male_nouns = DEFAULT_MALE_NOUNS
    female_nouns = DEFAULT_FEMALE_NOUNS
    n_grams = ngrams(sentences.split(), window)

    for grams in n_grams:
        pos = 1
        m = 0
        f = 0
        for w in grams:
            pos += 1
            if w not in data:
                data[w] = {"m": 0, "f": 0}

            if pos == int((window + 1) / 2):
                if w in male_nouns:
                    m = 1
                if w in female_nouns:
                    f = 1
                if m > 0:
                    for t in grams:
                        if t not in data:
                            data[t] = {"m": 0, "f": 0}
                        data[t]['m'] += 1
                if f > 0:
                    for t in grams:
                        if t not in data:
                            data[t] = {"m": 0, "f": 0}
                        data[t]['f'] += 1
    return data


def getGenderCounts(file):
    with open(file, 'r', encoding='utf-8') as fp:
        sentences = fp.read()
    tokens = sentences.split()
    maleCount = len([w for w in tokens if w in DEFAULT_MALE_NOUNS])
    femaleCount = len([w for w in tokens if w in DEFAULT_FEMALE_NOUNS])
    return maleCount, femaleCount

In [3]:
#occu
window = 10
MinCount = 10
txtlengths = [12000, 14000, 16000, 18000]

occupations = load_doc('occupations.txt').split('\n')
occupations += [getUpper(w) for w in occupations if len(w) > 0]

for txtlength in txtlengths:
    biasPList = []
    biasPCList = []
    scorePList = []
    scorePCList = []
    scoreDictList = []
    maleRatio = []
    global_count_female = 0
    df_per_txt = pd.DataFrame()
    data_dir = f"./5000-{txtlength}"
    print('\nPresent length:{}'.format(txtlength))

    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'{data_dir}/female/'
        else:
            txtDir = f'{data_dir}/male/'

        print("Processing gender: {}".format(txtDir))
        blogs_gender = os.listdir(txtDir)
        print("Files:", len(os.listdir(txtDir)))
        for i in tqdm(range(0, len(blogs_gender))):
            m = blogs_gender[i]
            maleCount = 0
            femaleCount = 0
            data = {}
            txt_path = txtDir + m
            data = get_cooccurrences(txt_path, data, window)
            mCount, fCount = getGenderCounts(txt_path)
            maleCount = mCount
            femaleCount = fCount
            scores_p, scores_pc, scores_dict = gender_ratios_m_f(
                data, maleCount, femaleCount)
            global_count_female += 1

            row_dict = dict()

            row_dict['word ratio'] = (maleCount + 0.000001) / (
                maleCount + femaleCount + 0.000001)
            row_dict['male'] = maleCount + 0.000001
            row_dict['female'] = femaleCount + 0.000001
            row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
            row_dict['biasPC'] = sum(scores_pc) / max(len(scores_pc), 1)
            row_dict['scores_dict'] = scores_dict
            row_dict['gender'] = gender

            df_per_txt = df_per_txt.append(row_dict, ignore_index=True)

    df_per_txt.to_csv(f'5000-{txtlength}_genderbias-occu.csv', index=False)

  0%|          | 1/2251 [00:00<05:13,  7.17it/s]


Present length:12000
Processing gender: ./5000-12000/female/
Files: 2251


  0%|          | 1/2140 [00:00<05:26,  6.55it/s]

Processing gender: ./5000-12000/male/
Files: 2140


  0%|          | 1/2213 [00:00<05:50,  6.30it/s]

...

  0%|          | 0/2153 [00:00<?, ?it/s]


Present length:18000
Processing gender: ./5000-18000/female/
Files: 2153


  0%|          | 0/2003 [00:00<?, ?it/s]

Processing gender: ./5000-18000/male/
Files: 2003


100%|██████████| 2003/2003 [06:34<00:00,  5.11it/s]


In [4]:
#emo
window = 10
MinCount = 10

occupations = load_doc('emotions.txt').split('\n')
occupations += [getUpper(w) for w in occupations if len(w) > 0]

for txtlength in txtlengths:
    biasPList = []
    biasPCList = []
    scorePList = []
    scorePCList = []
    scoreDictList = []
    maleRatio = []
    global_count_female = 0
    df_per_txt = pd.DataFrame()
    data_dir = f"./5000-{txtlength}"
    print('\nPresent length:{}'.format(txtlength))

    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'{data_dir}/female/'
        else:
            txtDir = f'{data_dir}/male/'

        print("Processing gender: {}".format(txtDir))
        blogs_gender = os.listdir(txtDir)
        print("Files:", len(os.listdir(txtDir)))
        for i in tqdm(range(0, len(blogs_gender))):
            m = blogs_gender[i]
            maleCount = 0
            femaleCount = 0
            data = {}
            txt_path = txtDir + m
            data = get_cooccurrences(txt_path, data, window)
            mCount, fCount = getGenderCounts(txt_path)
            maleCount = mCount
            femaleCount = fCount
            scores_p, scores_pc, scores_dict = gender_ratios_m_f(
                data, maleCount, femaleCount)
            global_count_female += 1

            row_dict = dict()

            row_dict['word ratio'] = (maleCount + 0.000001) / (
                maleCount + femaleCount + 0.000001)
            row_dict['male'] = maleCount + 0.000001
            row_dict['female'] = femaleCount + 0.000001
            row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
            row_dict['biasPC'] = sum(scores_pc) / max(len(scores_pc), 1)
            row_dict['scores_dict'] = scores_dict
            row_dict['gender'] = gender

            df_per_txt = df_per_txt.append(row_dict, ignore_index=True)

    df_per_txt.to_csv(f'5000-{txtlength}_genderbias-emo.csv', index=False)

  0%|          | 1/2251 [00:00<05:47,  6.48it/s]


Present length:12000
Processing gender: ./5000-12000/female/
Files: 2251


  0%|          | 1/2140 [00:00<04:43,  7.56it/s]

Processing gender: ./5000-12000/male/
Files: 2140


  0%|          | 1/2213 [00:00<05:55,  6.22it/s]

...

  0%|          | 1/2153 [00:00<06:57,  5.16it/s]


Present length:18000
Processing gender: ./5000-18000/female/
Files: 2153


  0%|          | 0/2003 [00:00<?, ?it/s]

Processing gender: ./5000-18000/male/
Files: 2003


100%|██████████| 2003/2003 [06:40<00:00,  5.03it/s]


In [5]:
#verb
window = 10
MinCount = 10

occupations = load_doc('verb.txt').split('\n')
occupations += [getUpper(w) for w in occupations if len(w) > 0]

for txtlength in txtlengths:
    biasPList = []
    biasPCList = []
    scorePList = []
    scorePCList = []
    scoreDictList = []
    maleRatio = []
    global_count_female = 0
    df_per_txt = pd.DataFrame()
    data_dir = f"./5000-{txtlength}"
    print('\nPresent length:{}'.format(txtlength))

    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'{data_dir}/female/'
        else:
            txtDir = f'{data_dir}/male/'

        print("Processing gender: {}".format(txtDir))
        blogs_gender = os.listdir(txtDir)
        print("Files:", len(os.listdir(txtDir)))
        for i in tqdm(range(0, len(blogs_gender))):
            m = blogs_gender[i]
            maleCount = 0
            femaleCount = 0
            data = {}
            txt_path = txtDir + m
            data = get_cooccurrences(txt_path, data, window)
            mCount, fCount = getGenderCounts(txt_path)
            maleCount = mCount
            femaleCount = fCount
            scores_p, scores_pc, scores_dict = gender_ratios_m_f(
                data, maleCount, femaleCount)
            global_count_female += 1

            row_dict = dict()

            row_dict['word ratio'] = (maleCount + 0.000001) / (
                maleCount + femaleCount + 0.000001)
            row_dict['male'] = maleCount + 0.000001
            row_dict['female'] = femaleCount + 0.000001
            row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
            row_dict['biasPC'] = sum(scores_pc) / max(len(scores_pc), 1)
            row_dict['scores_dict'] = scores_dict
            row_dict['gender'] = gender

            df_per_txt = df_per_txt.append(row_dict, ignore_index=True)

    df_per_txt.to_csv(f'5000-{txtlength}_genderbias-verb.csv', index=False)

  0%|          | 1/2251 [00:00<04:57,  7.55it/s]


Present length:12000
Processing gender: ./5000-12000/female/
Files: 2251


  0%|          | 1/2140 [00:00<04:51,  7.33it/s]

Processing gender: ./5000-12000/male/
Files: 2140


  0%|          | 1/2213 [00:00<05:37,  6.56it/s]

...

  0%|          | 0/2153 [00:00<?, ?it/s]


Present length:18000
Processing gender: ./5000-18000/female/
Files: 2153


  0%|          | 0/2003 [00:00<?, ?it/s]

Processing gender: ./5000-18000/male/
Files: 2003


100%|██████████| 2003/2003 [06:43<00:00,  4.78it/s]
